In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
# from scipy.sparse import csr_matrix

Using TensorFlow backend.
C:\Users\britt\miniconda3\envs\bobcat_env_36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\britt\miniconda3\envs\bobcat_env_36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\britt\miniconda3\envs\bobcat_env_36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\britt\minic

### Load data

In [8]:
test_boxes = pd.read_csv('tbl_test_boxes.csv')
train_boxes = pd.read_csv('tbl_train_boxes.csv')

In [5]:
unboxer_attributes = pd.read_csv('tbl_unboxer_attributes.csv')

C:\Users\britt\miniconda3\envs\bobcat_env_36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (61,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
unboxer_keywords = pd.read_csv('tbl_unboxer_keywords.csv')

In [ ]:
#vendor_keywords = pd.read_csv('tbl_vendor_keywords.csv')

### Look at the data 

In [ ]:
def check_for_nans_by_column(df):
    for col in df.columns:
        n_nans = len(df[df[col].isna()==True])
        print('{}: {} nans'.format(col, n_nans))

In [ ]:
# simple enough
#train_boxes

In [ ]:
check_for_nans_by_column(train_boxes)

In [ ]:
# same
#test_boxes

In [ ]:
check_for_nans_by_column(test_boxes)

In [ ]:
# will need to decide how to handle nans
# beartrap variables are categorical - will need to one-hot-encode

# it seems that skimask features can be 0, 1, or -1
# I assume these are meant to mean null, positive, or negative
# so then it would make sense to replace nulls with 0s
# then I could one hot encode 

#unboxer_attributes

In [ ]:
check_for_nans_by_column(unboxer_attributes)

In [ ]:
# categorical - simple enough
# need to switch enabled to 0s and 1s
#unboxer_keywords

In [ ]:
check_for_nans_by_column(unboxer_keywords)

In [ ]:
# again, need to deal with nans - how many rows are all nan?
# need to switch enabled to 0s and 1s
# vendor_keywords

In [ ]:
# so it seems that we can just discard any rows with a nan
check_for_nans_by_column(vendor_keywords)

### Clean the data 
Let's move backwards on this one

In [ ]:
'''

# drop rows with nans
vendor_keywords = vendor_keywords.dropna()

# replace -1s with 0s
vendor_keywords = vendor_keywords.replace(-1, 0)

# SUMBSAMPLE FOR NOW BECAUSE IM IMPATIENT: TODO: FIX THIS LATER
#print(len(vendor_keywords))
#vendor_keywords = vendor_keywords.sample(frac=0.01, random_state=42)
#print(len(vendor_keywords))

# need to one hot encode
vendor_keywords['keyword_id'] = vendor_keywords['keyword_id'].astype('category')
vendor_keywords = pd.get_dummies(vendor_keywords)

cols_to_multiply = vendor_keywords.columns.to_list()[2:]

'''

In [ ]:
# use numpy for faster multiplication
#product = vendor_keywords[cols_to_multiply].values * vendor_keywords['enabled'][:, np.newaxis]

In [ ]:
#print(product.shape)
#print(vendor_keywords['vendor_id'].values.shape)

In [ ]:
'''
# put it back into a dataframe
cols_to_multiply.insert(0, 'vendor_id')
arr = np.hstack((vendor_keywords['vendor_id'].values[:, np.newaxis], product))
vendor_keywords = pd.DataFrame(data=arr, columns=cols_to_multiply)
'''

In [ ]:
#vendor_keywords

In [ ]:
'''
# need to aggregate by user_id
print('aggregating...')
vendor_keywords_agg = vendor_keywords.groupby('vendor_id').sum()

vendor_keywords_agg
'''

In [ ]:
# save this so I don't have to do it again
# vendor_keywords_agg.to_csv('vendor_keywords_agg.csv')

In [2]:
vendor_keywords_agg = pd.read_csv('vendor_keywords_agg.csv')

In [ ]:
unboxer_keywords

In [ ]:
'''
# unboxer keywords now

# replace -1s with 0s
unboxer_keywords = unboxer_keywords.replace(-1, 0)

# SUMBSAMPLE FOR NOW BECAUSE IM IMPATIENT: TODO: FIX THIS LATER
#print(len(unboxer_keywords))
#unboxer_keywords = unboxer_keywords.sample(frac=0.5, random_state=42)
#print(len(unboxer_keywords))

# need to one hot encode
unboxer_keywords['keyword_id'] = unboxer_keywords['keyword_id'].astype('category')
unboxer_keywords = pd.get_dummies(unboxer_keywords)
cols_to_multiply = unboxer_keywords.columns.to_list()[2:]

print('multiplying...')

# use numpy for faster multiplication
product = unboxer_keywords[cols_to_multiply].values * unboxer_keywords['enabled'][:, np.newaxis]

print('converting back to df...')
# put it back into a dataframe
cols_to_multiply.insert(0, 'unboxer_id')
arr = np.hstack((unboxer_keywords['unboxer_id'].values[:, np.newaxis], product))
unboxer_keywords = pd.DataFrame(data=arr, columns=cols_to_multiply)
'''

In [ ]:
'''
print(unboxer_keywords.shape)
# need to aggregate by user_id
print('aggregating...')
unboxer_keywords_agg = unboxer_keywords.groupby('unboxer_id').sum()
unboxer_keywords_agg
'''

In [ ]:
# save this so I don't have to do it again
#unboxer_keywords_agg.to_csv('unboxer_keywords_agg.csv')

In [3]:
unboxer_keywords_agg = pd.read_csv('unboxer_keywords_agg.csv')

In [6]:
# unboxer attributes now 

# replace nans with 0s 
unboxer_attributes = unboxer_attributes.fillna(0)

# now everything can be one-hot-encoded
# going to have so many columns... can definitely remove correlated/insignificant ones later if needed

# first we need to make all of the columns categorical
def make_all_cols_categorical(df):
    for col in df.columns:
        if col != 'unboxer_id':
            df[col] = df[col].astype('category')
    return(df)
unboxer_attributes = make_all_cols_categorical(unboxer_attributes)
# do the one hot encoding
unboxer_attributes = pd.get_dummies(unboxer_attributes)

unboxer_attributes

,unboxer_id,feature_skimask1_-1.0,feature_skimask1_0.0,feature_skimask1_1.0,feature_skimask2_-1.0,feature_skimask2_0.0,feature_skimask2_1.0,feature_skimask3_-1.0,feature_skimask3_0.0,feature_skimask3_1.0,...,feature_beartrap5_91,feature_beartrap5_92,feature_beartrap5_93,feature_beartrap5_94,feature_beartrap5_95,feature_beartrap5_96,feature_beartrap5_97,feature_beartrap5_98,feature_beartrap5_99,feature_beartrap5_Turkey Contest Feature ID
0,1,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9022,9023,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9023,9024,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9024,9025,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9025,9026,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# test_boxes and train_boxes are fine for now

### Combine the tables to make one big dataset

In [9]:
# let me subsample to make things run quicky for now
# when I have a model I can remove insignificant/correlated features and use the whole dataset
train_boxes_subsampled = train_boxes.sample(frac=.4, random_state=42)

In [ ]:
#train_boxes_subsampled

In [ ]:
print('So I will expect to see maximum {} rows throughout this whole merging process'.format(len(train_boxes_subsampled)))

In [ ]:
print(len(unboxer_keywords_agg), len(train_boxes_subsampled), len(unboxer_keywords_agg)*len(train_boxes_subsampled))

In [10]:
# join the unboxer keywords

# left.merge(right, on='user_id', how='left')
train_full_data = pd.merge(train_boxes_subsampled, unboxer_keywords_agg, on='unboxer_id', how='left')
# TODO: remove this later
# since I sampled the unboxer_keywords df, there are some unboxers which are not represented. 
# Drop their rows for now
# train_full_data = train_full_data.dropna()
# train_full_data

In [11]:
# sanity check- no nans in input?
def check_any_nans(df):
    return df.isnull().any() 
'''
print(check_any_nans(train_boxes_subsampled))
print('\n')
print(check_any_nans(unboxer_keywords_agg))
print('\n')
print(check_any_nans(train_full_data))
print('\n')
print(len(train_full_data.dropna())) # so there are not many nan's... let me just drop those rows for now
'''

train_full_data = train_full_data.dropna()

In [12]:
# join the vendor keywords 
train_full_data = pd.merge(train_full_data, vendor_keywords_agg, on='vendor_id', how='left')
#train_full_data

In [13]:
print(check_any_nans(train_full_data))
print('\n')
print(len(train_full_data.dropna())) 
# drop nans again
train_full_data = train_full_data.dropna()

unboxer_id           False
vendor_id            False
target_bobcat        False
keyword_id_1         False
keyword_id_2         False
                     ...  
keyword_id_2368.0     True
keyword_id_2369.0     True
keyword_id_2370.0     True
keyword_id_2371.0     True
keyword_id_2372.0     True
Length: 4605, dtype: bool


38252


In [14]:
# join unboxer attributes
train_full_data = pd.merge(train_full_data, unboxer_attributes, on='unboxer_id', how='left')
# train_full_data

In [15]:
print(check_any_nans(train_full_data))
print('\n')
print(len(train_full_data.dropna())) 
# drop nans again
train_full_data = train_full_data.dropna()

unboxer_id                                     False
vendor_id                                      False
target_bobcat                                  False
keyword_id_1                                   False
keyword_id_2                                   False
                                               ...  
feature_beartrap5_96                            True
feature_beartrap5_97                            True
feature_beartrap5_98                            True
feature_beartrap5_99                            True
feature_beartrap5_Turkey Contest Feature ID     True
Length: 5170, dtype: bool


38232


In [16]:
# save this so I don't have to do it again
train_full_data.to_pickle('train_full_data.pkl')

In [17]:
train_full_data

,unboxer_id,vendor_id,target_bobcat,keyword_id_1,keyword_id_2,keyword_id_3,keyword_id_4,keyword_id_5,keyword_id_6,keyword_id_7,...,feature_beartrap5_91,feature_beartrap5_92,feature_beartrap5_93,feature_beartrap5_94,feature_beartrap5_95,feature_beartrap5_96,feature_beartrap5_97,feature_beartrap5_98,feature_beartrap5_99,feature_beartrap5_Turkey Contest Feature ID
0,6587,59054,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2316,37909,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3460,44994,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4255,59347,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7697,22306,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38247,6021,51596,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38248,8143,35617,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38249,7131,64757,0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38250,2992,5936,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert dataframe to features and targets arrays, split train and test, deal with unbalanced classes

In [18]:
# I need to deal with unbalanced classes- we want an equal number of bobcats and not bobcats

# Separate majority and minority classes
df_majority = train_full_data[train_full_data.target_bobcat==0]
df_minority = train_full_data[train_full_data.target_bobcat==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,                  # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42)               # reproducible results
 
# Combine majority class with upsampled minority class
train_full_data = pd.concat([df_majority, df_minority_upsampled])

# remove the unneccessary columns
train_full_data = train_full_data.drop(columns=['unboxer_id', 'vendor_id'])

In [19]:
# separate into train and test sets
train, test = train_test_split(train_full_data, test_size=0.2)

In [20]:
# split features from targets, convert to array
x_train = train.drop(columns='target_bobcat').values
y_train = train['target_bobcat'].values
x_test = test.drop(columns='target_bobcat').values
y_test = test['target_bobcat'].values

In [21]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(56011, 5167) (56011,) (14003, 5167) (14003,)


### Define and train a model 
If 7.5% of boxes contain a bobcat, then if my model is choosing randomly it would get accuracy=0.075. This is the number to beat for a performant model. 

In [22]:
model = Sequential()
# start with something very simple
model.add(Dense(500, input_dim=x_train.shape[1], activation='relu', kernel_initializer='he_uniform'))  # input layer
model.add(Dense(200, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # sigmoid to choose 1 or 0

In [23]:
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)
model.summary()


Train on 56011 samples, validate on 14003 samples
Epoch 1/3
56011/56011 [==============================] - 29s 521us/step - loss: 0.5109 - accuracy: 0.7382 - val_loss: 0.4138 - val_accuracy: 0.8063
Epoch 2/3
56011/56011 [==============================] - 29s 515us/step - loss: 0.3761 - accuracy: 0.8308 - val_loss: 0.3196 - val_accuracy: 0.8628
Epoch 3/3
56011/56011 [==============================] - 34s 607us/step - loss: 0.3076 - accuracy: 0.8675 - val_loss: 0.2978 - val_accuracy: 0.8783
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               2584000   
_________________________________________________________________
dense_2 (Dense)              (None, 200)               100200    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 2,684,4

### Evaluate the model  